<h1> Gordon Growth Model / Dividend Discount Model Calculator

The Gordon Growth Model (GGM) aka Divident Discount Model is a tool used to determine the value of a stock based on a series of dividends. The GGM is commonly used to value stocks based on the net present values of their future dividends.




import requests
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import datetime


<h1> Assumptions
    

We assume the growth rate is constant as years progress
Cost of equity is bigger than divident growth (in order to get a postive evaluation)


<h1> Formulas & Calculations

Gordon Growth Rate Model  = (Dividend)*(1+growth rate) /(cost of equity- growth rate)
Growth Rate = Return on Equity(ROE) * (1 — Dividend payout ratio)

cost of equity = Risk free rate + Beta  (market return — Risk free rate)

yearly return = (end balance / start balance) -1


In [72]:
def comp_val(quote):
    div= requests.get(f'https://financialmodelingprep.com/api/v3/financials/income-statement/{quote}')
    div=div.json()
    divTD=float(div['financials'][0]['Dividend per Share'])
    print(F'{quote} Dividend per Share is {round(divTD,2)}')
    
    metrics=requests.get(f'https://financialmodelingprep.com/api/v3/company-key-metrics/{quote}')
    metrics =metrics.json()
    ROE = float(metrics['metrics'][0]['ROE'])
    payout_ratio = float(metrics['metrics'][0]['Payout Ratio'])
    sustainable_growth_rate =ROE *(1-payout_ratio)
    print(f'The Sustainable Growth Rate for {quote} is {round(sustainable_growth_rate*100,2)}%')
    
    start = datetime.datetime(2019, 2, 1)
    end = datetime.datetime(2020, 2, 27)
    
    treasury_values=web.DataReader(['TB1YR'],'fred',start,end)
    #print(treasury_values)
    Risk_Free_rate=float(treasury_values.iloc[-1])
    Risk_Free_rate =Risk_Free_rate/100
    #used  1-year Treasury Bills as a examplar due to Treasury bills are risk free
    print(f'The Risk Free Rate for {quote} is {Risk_Free_rate*100}%')
    
    beta = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{quote}')
    beta=beta.json()
    beta =float(beta['profile']['beta'])
    print(f'The beta for {quote} is {round(beta,2)}')
    
    #s&P 500 used as an examplar for market_return used for expected market return
    start = datetime.datetime(2019, 2, 15)
    end =datetime.datetime(2020,2,15)
    sp500 = web.DataReader(['sp500'],'fred',start,end)
    sp500.dropna(inplace = True)

    sp500_yearly_return= float((sp500.iloc[-1]/sp500.iloc[0])-1)
    #print(sp500)
    print(f'The yearly market return was around {round(sp500_yearly_return*100,2)}%')
    
    cost_equity = Risk_Free_rate + (beta*(sp500_yearly_return-Risk_Free_rate))
    print(f'The cost equity for {quote} is  {round(cost_equity*100,2)}%')
    
    GGM = (divTD*(1+sustainable_growth_rate))/((cost_equity-sustainable_growth_rate))
    
    return GGM
quote =input("Enter the stock name: ") 
    
print(f'The value of {quote} using the Gordon Growth model is {comp_val(quote)}')

Enter the stock name: SBUX
SBUX Dividend per Share is 1.43
The Sustainable Growth Rate for SBUX is -29.57%
The Risk Free Rate for SBUX is 1.49%
The beta for SBUX is 0.5
The yearly market return was around 21.78%
The cost equity for SBUX is  11.72%
The value of SBUX using the Gordon Growth model is 2.4363178193440143


<h1> Things to Consider

The Gordon Growth model is very sensitive to the inputs, any slight change to the growth rate, cost of equity and etc will have a big effect on the calculated value. It tends to work well with stocks that tend to me consitent and constant.
 
This model will not work with companies that do not pay dividends ( we also assume constant dividend payments which is also not very realistic)